# criar um Navegador
# importar/visualizar a base de dados

# Passo a passo

# Entra no google
# Pesquisar pelo produto
# Entra na aba shopping
# Pegar as informações do produto

# para cada item dentro da nossa base de dados, para cada produto

    # Procura esse produto no google shopping
        # Verificar se algum dos produtos do google shopping está dentro da minha faixa de preço

    # Procurar esse produto no Buscape
        # Verificar se algum dos produtos do buscape está dentro da minha faixa de preço

# Salvar as ofertas boas em um dataframe (tabela)
# Exportar pro excel
# enviar por email o resultado da tabela

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configurando as opções do Brave
chrome_options = Options()
chrome_options.binary_location = r"C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe"

# Configurando o serviço do driver
servico = Service(ChromeDriverManager().install())

# Iniciando o navegador com as opções configuradas
navegador = webdriver.Chrome(service=servico, options=chrome_options)

# Carregar a tabela de produtos
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)


,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,1200,2500
1,rtx 3060,zota galax,1500,3000


In [2]:
def verificar_termos_banidos(lista_termos_banidos, nome):
    # analisar se ele não tem nenhum termo banido
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_todos_termos_produto(lista_termos_nomes_produtos, nome):
    # analisar se ele tem todos os termos do nome do produto
    tem_todo_termos_produtos = True
    for palavra in lista_termos_nomes_produtos:
        if palavra not in nome:
            tem_todo_termos_produtos = False
    return tem_todo_termos_produtos

def buscar_bing_compras(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
    # Entra no site do Bing
    navegador.get("https://www.bing.com/")
    time.sleep(3)
    if navegador.find_elements('xpath', '//*[@id="bnp_btn_accept"]'):
        # Aceita os cookies se o botão estiver presente
        print("Aceitando os cookies do Bing...")
        # Clica no botão de aceitar cookies
        navegador.find_element('xpath', '//*[@id="bnp_btn_accept"]/a').click()
        time.sleep(1)
    else:
        pass

    # Produto a ser pesquisado na parte de Compras do Bing
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nomes_produtos = produto.split(" ")
    lista_de_ofertas_mercadolivre = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    navegador.find_element('xpath', '//*[@id="sb_form_q"]').send_keys(produto, Keys.ENTER)
    time.sleep(1)
    elemento = navegador.find_element("xpath", '//*[@id="b-scopeListItem-shop"]/a').click()

    # Valor dos produtos
    time.sleep(2)
    # Troca para a nova aba (índice 1)
    nova_aba = navegador.window_handles[1]
    navegador.switch_to.window(nova_aba)

    # Troca para a aba anterior (índice 0), fecha ela e volta para a nova aba
    aba_anterior = navegador.window_handles[0]
    navegador.switch_to.window(aba_anterior)
    navegador.close()

    # Volta para a nova aba (índice 1)
    navegador.switch_to.window(nova_aba)
 
    time.sleep(2)
    lista_resultados = navegador.find_elements('class name', 'br-gOffCard')
    print(lista_resultados)

    for resultado in lista_resultados:
        nome = resultado.find_element('class name', "br-offTtl").text
        nome = nome.lower()

        # analisar se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_termos_banidos(lista_termos_banidos, nome)

        # analisar se ele tem todos os termos do nome do produto
        tem_todo_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nomes_produtos, nome)
        
        # selecionar só os elementos que tem_termos_produto = False e ao mesmo tempo tem_todos_termos_produtos = True
        if not tem_termos_banidos and tem_todo_termos_produtos:
            time.sleep(1)
            preco_div = resultado.find_element('class name', 'br-offPrice')
            filhos = preco_div.find_elements('xpath', './*')
            
            preco = None
            classes_desejadas = ['l2vh_pr', 'pd-price', 'br-standardPrice']

            for filho in filhos:
                classes = filho.get_attribute('class').split()
                if any(classe in classes for classe in classes_desejadas):
                    preco = filho.text
                    break

            preco = preco.replace("R$", "").replace("", "").replace(".", "").replace(",", ".").strip()
            preco = float(preco)

            # se o preco está entre o mínimo e o máximo
            if preco_minimo <= preco <= preco_maximo:
                # extrair o link do produto
                link = resultado.find_element('class name', 'br-offLink').get_attribute('href')
                lista_de_ofertas_mercadolivre.append((nome, preco, link)) 

    return lista_de_ofertas_mercadolivre
                    

def busca_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
    # trata os valores
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nomes_produtos = produto.split(" ")
    lista_de_ofertas_buscape = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    # entra no site do Buscapé
    navegador.get("https://www.buscape.com.br/")
    time.sleep(2)
    navegador.find_element('xpath', '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)

    # pegar os resultados
    lista_resultados = navegador.find_elements('class name', 'ProductCard_ProductCard_Inner__gapsh')

    time.sleep(1)
    for resultado in lista_resultados:
        preco = resultado.find_element('class name', 'Text_MobileHeadingS__HEz7L').text
        nome = resultado.find_element('class name', 'ProductCard_ProductCard_Name__U_mUQ').text
        nome = nome.lower()
        link = resultado.get_attribute('href')
        

        # analisar se ele não tem nenhum termo banido
        tem_termos_banidos = verificar_termos_banidos(lista_termos_banidos, nome)

        # analisar se ele tem todos os termos do nome do produto
        tem_todo_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nomes_produtos, nome)
        
    # analisar se o preço está entre o preço mínimo e o preço máximo
        if not tem_termos_banidos and tem_todo_termos_produtos:
            preco = preco.replace("R$", "").replace("", "").replace(".", "").replace(",", ".").strip()
            preco = float(preco)
            if preco_minimo <= preco <= preco_maximo:
                lista_de_ofertas_buscape.append((nome, preco, link))
    # retornar a lista de ofertas do buscapé
    return lista_de_ofertas_buscape


In [3]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]

    lista_ofertas_bing_compras = buscar_bing_compras(navegador, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_bing_compras:
        # Criar DataFrame e exibir tabela
        tabela_bing_compras = pd.DataFrame(lista_ofertas_bing_compras, columns=['produto', 'preço', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_bing_compras])
    else:
         tabela_bing_compras = None   

    lista_ofertas_buscape = busca_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape_compras = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preço', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape_compras])
    else:
        tabela_buscape_compras = None

navegador.quit()

display(tabela_ofertas)

Aceitando os cookies do Bing...
[<selenium.webdriver.remote.webelement.WebElement (session="19f41a27389b91297881d4fed5447a24", element="f.19A34346A6E5FF417660B8D0E1E7DB6D.d.96B219646D21D7E03CBBAC3640797C40.e.22")>, <selenium.webdriver.remote.webelement.WebElement (session="19f41a27389b91297881d4fed5447a24", element="f.19A34346A6E5FF417660B8D0E1E7DB6D.d.96B219646D21D7E03CBBAC3640797C40.e.23")>, <selenium.webdriver.remote.webelement.WebElement (session="19f41a27389b91297881d4fed5447a24", element="f.19A34346A6E5FF417660B8D0E1E7DB6D.d.96B219646D21D7E03CBBAC3640797C40.e.24")>, <selenium.webdriver.remote.webelement.WebElement (session="19f41a27389b91297881d4fed5447a24", element="f.19A34346A6E5FF417660B8D0E1E7DB6D.d.96B219646D21D7E03CBBAC3640797C40.e.25")>, <selenium.webdriver.remote.webelement.WebElement (session="19f41a27389b91297881d4fed5447a24", element="f.19A34346A6E5FF417660B8D0E1E7DB6D.d.96B219646D21D7E03CBBAC3640797C40.e.26")>, <selenium.webdriver.remote.webelement.WebElement (session

,produto,preço,link
0,apple iphone 12 (64 gb) - branco,2299.00,https://www.bing.com/aclick?ld=e8pjJXo5fD3FyBj...
1,apple iphone 12 (64 gb) - azul,2299.00,https://www.bing.com/aclick?ld=e8NA3AixJSuEoWx...
2,iphone 12 64 gb azul - excelente (recondici…,1899.00,https://www.bing.com/aclick?ld=e8cc3zyKBOHfYkE...
3,iphone 12 64 gb preto - aceitável (recondi…,1799.00,https://www.bing.com/aclick?ld=e8zDJAjYghfFKkV...
4,apple iphone 12 (64 gb) - roxo,2299.00,https://www.bing.com/aclick?ld=e8fKycsROjiU89c...
5,apple iphone 12 (64 gb) - verde,2299.00,https://www.bing.com/aclick?ld=e8vOiQHv1Z3OK9v...
6,iphone 12 64 gb (product)red - excel…,1800.08,https://www.bing.com/aclick?ld=e8ya_WQMvT1OVHI...
0,celular apple iphone 12 vermelho usado 64gb,2005.15,https://www.buscape.com.br/celular/smartphone-...
1,celular apple iphone 12 usado 64gb 4 gb,1750.56,https://www.buscape.com.br/celular/smartphone-...
0,placa de video geforce rtx 3060 12gb gddr6 msi...,2699.00,https://www.buscape.com.br/placa-de-video/plac...


In [4]:
## Salvar a tabela de ofertas em um arquivo Excel

tabela_ofertas.to_excel("tabela_ofertas.xlsx", index=False)

In [5]:
## inviar o arquivo Excel por e-mail
import win32com.client as win32

print("tentando enviar o e-mail...")
try:
    if len(tabela_ofertas) > 0:
        outlook = win32.Dispatch('outlook.application')
        mail = outlook.CreateItem(0)
        mail.To = 'bettencourtrafael01@gmail.com'
        mail.Subject = 'Tabela de Ofertas'
        mail.HTMLBody = f"""
        <p> Prezados,</p>
        <p> Encontramos alguns produtos em oferta dentro da faixa de preço desejada</p>
        {tabela_ofertas.to_html(index=False)}
        <p>Att,</p>
        """
        attachment = r'tabela_ofertas.xlsx'
        mail.Send()
        print("E-mail enviado com sucesso!")
except Exception as e:
    print(f"Erro ao enviar o e-mail: {e}")

tentando enviar o e-mail...
E-mail enviado com sucesso!
